In [29]:
oracle_prices = {"WEST":0.04, "EAST":1, "USD":1, "BTC":1000}

from collections import defaultdict

tokens = {
    "sUSD":{"supply":0, "price":oracle_prices["USD"], "balances":defaultdict(int)},
    "sBTC":{"supply":0, "price":oracle_prices["BTC"], "balances":defaultdict(int)},
    "iBTC":{"supply":0, "price":1 / oracle_prices["BTC"], "balances":defaultdict(int)},
    }

def pull_prices_from_oracle():
    tokens["sUSD"]["price"] = oracle_prices["USD"]
    tokens["sBTC"]["price"] = oracle_prices["BTC"]
    tokens["iBTC"]["price"] = 1 / oracle_prices["BTC"]

def system_debt():
    s = 0
    for t in tokens:
        s += tokens[t]["supply"] * tokens[t]["price"]
    return s

debt_share_supply = 0 # sum of all debt shares

class UserPosition:
    
    def __init__(self, address):
        self.balance_of_east = 0
        self.balance_of_west = 0
        self.debt_share = 0
        #self.sbalances = {}
        self.address = address

    def deposit_collateral(self, east, west):
        assert (east != 0 or west != 0)
        self.balance_of_east += east
        self.balance_of_west += west

    @property
    def value_of_collateral(self):
        return self.balance_of_east * oracle_prices["EAST"] + 0.75 * self.balance_of_west * oracle_prices["WEST"] # 75% coefficient for volatile token

    def mint_susd(self, amount):
        global system_debt
        global debt_share_supply
        global tokens
        if self.value_of_collateral / amount < 3:
            raise "Can't mint, CRatio will be under 300%"
        else:
            if system_debt() == 0:
                self.debt_share = 1.0
                debt_share_supply = 1.0
            else:
                self.debt_share += debt_share_supply * amount / system_debt()
                debt_share_supply += debt_share_supply * amount / system_debt()
            tokens["sUSD"]["supply"] += amount
            tokens["sUSD"]["balances"][self.address] += amount

    @property
    def debt(self):
        global system_debt
        global debt_share_supply
        return self.debt_share / debt_share_supply * system_debt()

    @property
    def c_ratio(self):
        if self.debt == 0:
            return None
        else:
            return self.value_of_collateral / self.debt

    def burn_susd(self, amount):
        global system_debt
        global debt_share_supply
        global tokens
        if tokens["sUSD"]["balances"][self.address] >= amount:
            removed_debt_shares = debt_share_supply * amount / system_debt()
            tokens["sUSD"]["balances"][self.address] -= amount
            tokens["sUSD"]["supply"] -= amount
            self.debt_share -= removed_debt_shares
            debt_share_supply -= removed_debt_shares
        else:
            raise "Not enough sUSD in wallet"

    def withdraw_collateral(self, east, west):
        assert (east != 0 or west != 0)
        if ((self.balance_of_west - west) * oracle_prices["WEST"] * 0.75 + (self.balance_of_east - east) * oracle_prices["EAST"]) / self.debt < 3:
            raise "Can't withdraw this much, CRatio will be under 300%"
        else:
            self.balance_of_west -= west
            self.balance_of_east -= east

    def swap_synth(self, token1, amt1, token2): # sUSD 100 -> sBTC 
        global tokens
        if tokens[token1]["balances"][self.address] >= amt1:
            bought_amount = amt1 * tokens[token1]["price"] / tokens[token2]["price"]
            tokens[token2]["balances"][self.address] += bought_amount
            tokens[token1]["balances"][self.address] -= amt1
            tokens[token2]["supply"] += bought_amount
            tokens[token1]["supply"] -= amt1
        else:
          raise "Not enough tokens"

    def liquidate(self, liquidator, susd_amount):
        global debt_share_supply
        global tokens
        if susd_amount > tokens['sUSD']['balances'][liquidator]:
            raise "Not enough sUSD"
        if self.c_ratio <= 1.5:
            liquidation_penalty = 0.1 # PENALTY 10% -- 1% to liquidator, 9% to treasury
            needed_susd = (3 * self.debt - self.value_of_collateral) / (3 - (1 + liquidation_penalty)) # How much sUSD is needed to bring CRatio back to 300%
            liquidator_reward_share = needed_susd * (1 + liquidation_penalty) / self.value_of_collateral
            if susd_amount >= needed_susd:
                print(f"LIQUIDATION: {needed_susd} burned, {liquidator_reward_share * self.value_of_collateral}$ of {self.value_of_collateral}$ of collateral liquidated")
                removed_debt_shares = debt_share_supply * needed_susd / system_debt()
                tokens['sUSD']['balances'][liquidator] -= needed_susd
                tokens["sUSD"]["supply"] -= needed_susd
                self.balance_of_east -= liquidator_reward_share * self.balance_of_east # send 101/110 of it to liquidator, 9/110 to treasury
                self.balance_of_west -= liquidator_reward_share * self.balance_of_west # send 101/110 of it to liquidator, 9/110 to treasury
                self.debt_share -= removed_debt_shares
                debt_share_supply -= removed_debt_shares
            else:
                raise "Not enough sUSD to liquidate"
        else:
            raise "CRatio is healthy, can't liquidate"

    def print(self):
        global system_debt
        global tokens
        print("Staked east:", self.balance_of_east)
        print("Staked west:", self.balance_of_west)
        print("Owned synths:")
        sval = 0
        for t in tokens:
            print(" ",t, tokens[t]["balances"][self.address])
            sval += tokens[t]["balances"][self.address] * tokens[t]["price"]
        print("Owned synths value:", sval)
        print("Debt:", self.debt)
        #print("Debt share:", self.debt_share)
        print("CRatio:", self.c_ratio)
        if self.c_ratio<1.5: print("CAN BE LIQUIDATED")

In [30]:
users = {"address1":UserPosition("address1"), "address2":UserPosition("address2")}

In [31]:
users["address1"].balance_of_east += 250000
users["address1"].mint_susd(50000)
users["address1"].swap_synth("sUSD",50000,"sBTC")
users["address1"].print()

Staked east: 250000
Staked west: 0
Owned synths:
  sUSD 0
  sBTC 50.0
  iBTC 0
Owned synths value: 50000.0
Debt: 50000.0
CRatio: 5.0


In [32]:
users["address2"].balance_of_east += 250000
users["address2"].mint_susd(50000)
users["address2"].print()

Staked east: 250000
Staked west: 0
Owned synths:
  sUSD 50000
  sBTC 0
  iBTC 0
Owned synths value: 50000.0
Debt: 50000.0
CRatio: 5.0


In [33]:
oracle_prices["BTC"] = 8000
pull_prices_from_oracle()
users["address1"].print()
print()
users["address2"].print()

Staked east: 250000
Staked west: 0
Owned synths:
  sUSD 0
  sBTC 50.0
  iBTC 0
Owned synths value: 400000.0
Debt: 225000.0
CRatio: 1.1111111111111112
CAN BE LIQUIDATED

Staked east: 250000
Staked west: 0
Owned synths:
  sUSD 50000
  sBTC 0
  iBTC 0
Owned synths value: 50000.0
Debt: 225000.0
CRatio: 1.1111111111111112
CAN BE LIQUIDATED


In [34]:
users["address2"].balance_of_east += 2500000
users["address2"].mint_susd(500000)
users["address1"].liquidate(liquidator = "address2", susd_amount=300000)
print()
users["address1"].print()
print()
users["address2"].print()

LIQUIDATION: 223684.2105263158 burned, 246052.6315789474$ of 250000.0$ of collateral liquidated

Staked east: 3947.3684210526117
Staked west: 0.0
Owned synths:
  sUSD 0
  sBTC 50.0
  iBTC 0
Owned synths value: 400000.0
Debt: 1315.789473684198
CRatio: 3.0000000000000138

Staked east: 2750000
Staked west: 0
Owned synths:
  sUSD 326315.7894736842
  sBTC 0
  iBTC 0
Owned synths value: 326315.7894736842
Debt: 725000.0
CRatio: 3.793103448275862
